In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
col_order = ["pt", "e", "eta", "phi"]
reco = pd.read_csv("../reco.csv", index_col=0)[col_order + ["type", "idx_original"]].values
gen = pd.read_csv("../gen.csv", index_col=0)[col_order + ["pid", "type", "idx_original", "idx_original2"]].values

In [ ]:
plt.hist(gen[:, 0], bins=np.linspace(0,5,101));
plt.yscale("log")

In [ ]:
plt.hist(gen[:, 1], bins=np.linspace(0,100,101));
plt.yscale("log")

In [ ]:
len(gen), np.sum(gen[:, 0]>0.2)

In [ ]:
fi = np.load("../ev.npz")

In [ ]:
arr = fi["reco_to_gen"]
arr2 = (arr.T / reco[:, 1]).T
arr3 = arr2 > 0.0

In [ ]:
plt.hist(arr[arr!=0], bins=100);
plt.yscale("log")

In [ ]:
plt.hist(arr2[arr2!=0].flatten(), bins=np.linspace(0.8,1.2, 100));

In [ ]:
import matplotlib
plt.figure(figsize=(20,20))
plt.imshow(arr, cmap="Greys", norm=matplotlib.colors.LogNorm(vmin=0.5, vmax=100))

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(arr>0.0, cmap="Greys")

In [ ]:
plt.hist(arr3.sum(axis=0), bins=np.linspace(0,40,41), histtype="step", lw=2, density=1.0, label="gen");
plt.hist(arr3.sum(axis=1), bins=np.linspace(0,40,41), histtype="step", lw=2, density=1.0, label="reco");
plt.yscale("log")
plt.legend()

In [ ]:
highest_pt_idx = np.argsort(gen[:, 0])[::-1]

In [ ]:
remaining_indices = np.ones(arr.shape[0], dtype=np.int64)
pairs_reco_gen = {}
unreconstructable = []
for igen in highest_pt_idx:
    if gen[igen, 1] < 1 or gen[igen, 0]<0.2:
        continue
    best_reco_idx = np.argmax(arr[:, igen])
    if best_reco_idx != 0 and temp.sum()>0.0:
        if not (best_reco_idx in pairs_reco_gen):
            pairs_reco_gen[best_reco_idx] = []
        if arr[best_reco_idx, igen] / reco[best_reco_idx, 1] > 0.2:
            pairs_reco_gen[best_reco_idx] += [(igen, arr[best_reco_idx, igen])]
    else:
        unreconstructable += [igen]

pairs_reco_gen_sorted = {}
for k, v in pairs_reco_gen.items():
    v = sorted(v, key=lambda x: x[1], reverse=True)
    pairs_reco_gen_sorted[k] = v

pairs_reco_gen = pairs_reco_gen_sorted

In [ ]:
len(gen), len(pairs_reco_gen), len(reco)

In [ ]:
for ireco in range(len(reco)):
    if not ireco in pairs_reco_gen:
        pairs_reco_gen[ireco] = []

In [ ]:
gen[0]

In [ ]:
def merge_gen(gens):
    if len(gens) == 1:
        ret = gen[gens[0]]
    else:
        inds_gen = np.array([g[0] for g in gens])
        ret = gen[inds_gen]
        nparticles = ret.shape[0]
        
        ret2 = ret.sum(axis=0)
        ret2[2] = ret2[2]/nparticles
        ret2[3] = ret2[3]/nparticles
        
        ret2[4] = 1
    return ret2

In [ ]:
pairs_reco_gen[2185]

In [ ]:
gen[3757]

In [ ]:
merge_gen(pairs_reco_gen[2185])

In [ ]:
for ireco in range(len(reco)):
    print(ireco, reco[ireco, 2], reco[ireco, 3], len(pairs_reco_gen[ireco]))

In [ ]:
plt.hist([len(x[1]) for x in pairs_reco_gen.items()], np.linspace(0,10,11));

In [ ]:
ir = 6603
print("rec", ir, ", ".join(["{:.4f}".format(x) for x in reco[ir]]))
for ig, cmp in pairs_reco_gen.get(ir, []):
    print("gen", ig, ", ".join(["{:.4f}".format(x) for x in gen[ig]]), round(cmp, 4))

In [ ]:
import uproot

In [ ]:
fi = uproot.open("../pfntuple.root")

In [ ]:
#fi.get("ana/pftree").keys()

In [ ]:
sh = fi.get("ana/pftree").array(b'simhit_idx_simcluster')[0]
sh_detid = fi.get("ana/pftree").array(b'simhit_detid')[0]
sh_frac = fi.get("ana/pftree").array(b'simhit_frac')[0]

rh = fi.get("ana/pftree").array(b'rechit_idx_cluster')[0]
rh_detid = fi.get("ana/pftree").array(b'rechit_detid')[0]
rh_e = fi.get("ana/pftree").array(b'rechit_e')[0]


rh_d = {}
for i in range(len(rh_detid)):
    k = rh_detid[i]
    if not (k in rh_d):
        rh_d[k] = 0.0
    rh_d[k] += rh_e[i]

In [ ]:
# rec 5711 0.0000, 22.0067, -3.9391, 3.0543, 0.0000, 4167.0000
# gen 5566 0.7082, 17.0231, -3.8723, 2.9825, 22.0000, 0.0000, 6019.0000, -1.0000 7.3355
# gen 5552 0.7022, 25.5821, -4.2884, 2.3115, -211.0000, 1.0000, 130796.0000, 10933.0000 7.3355

In [ ]:
msk0 = np.nonzero(rh == 4167)[0]
msk = np.nonzero(sh == 10933)[0]
msk2 = np.nonzero(sh == 6019)[0]

In [ ]:
ls = []
se = 0
for i in zip(rh_detid[msk0], rh_e[msk0]):
    if i[0] in ls:
        print("dupe")
    ls += [i[0]]
    se += i[1]
    print(i)

In [ ]:
tot = 0.0
for f, d in zip(sh_frac[msk], sh_detid[msk]):
    x = rh_d.get(d, 0.0)*f
    tot += x
    print(f, d, x)

In [ ]:
tot = 0.0
for f, d in zip(sh_frac[msk2], sh_detid[msk2]):
    x = rh_d.get(d, 0.0)*f
    tot += x
    print(f, d, x)

In [ ]:
pairs_reco_gen_l = list(pairs_reco_gen.items())

In [ ]:
pairs_reco_gen_l = sorted(pairs_reco_gen_l, key=lambda x: len(x[1]), reverse=True)

In [ ]:
ks = [x[0] for x in pairs_reco_gen_l]

In [ ]:
for k in ks:
    ir = k
    print("rec", ir, ", ".join(["{:.4f}".format(x) for x in reco[ir]]))
    for ig, cmp in pairs_reco_gen.get(ir, []):
        print("gen", ig, ", ".join(["{:.4f}".format(x) for x in gen[ig]]), round(cmp, 4))
    print("--")

In [ ]:
Xs = []
ys = []
ycs = []
for i in range(8):
    fi = np.load("../ev_{}.npz".format(i))
    X = fi["X"]
    y = fi["ygen"]
    yc = fi["ycand"]
    
    Xs += [X]
    ys += [y]
    ycs += [yc]

In [ ]:
rg = fi["reco_gen"]
rc = fi["reco_cand"]

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(rg>0.0, cmap="Greys")
plt.xlabel("genparticle index")
plt.ylabel("recoparticle index")

In [ ]:
rgs = rg>0.0
plt.hist(rgs.sum(axis=0), bins=np.linspace(0, 20, 21), density=1.0, histtype="step", lw=2, label="gen");
plt.hist(rgs.sum(axis=1), bins=np.linspace(0, 20, 21), density=1.0, histtype="step", lw=2, label="reco");
plt.legend()
#plt.yscale("log")
plt.xlabel("number of associations")
plt.ylabel("fraction of total")

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(rc>0.0, cmap="Greys")

In [ ]:
rcs = rc>0.0
plt.hist(rcs.sum(axis=0), bins=np.linspace(0, 10, 11), density=1.0, histtype="step", lw=2, label="candidate");
plt.hist(rcs.sum(axis=1), bins=np.linspace(0, 10, 11), density=1.0, histtype="step", lw=2, label="reco");
plt.legend()
#plt.yscale("log")
plt.xlabel("number of associations")
plt.ylabel("fraction of total")

In [ ]:
X = np.vstack(Xs)
y = np.vstack(ys)
yc = np.vstack(ycs)

In [ ]:
plt.figure(figsize=(8,4))
plt.suptitle("eta")

plt.subplot(1,2,1)
m = y[:, 0] != 0
m2 = yc[:, 0] != 0

plt.scatter(X[m, 2], y[m, 2], marker=".", alpha=0.2)
plt.plot([-5, 5], [-5, 5], color="black")
plt.xlabel("reco")
plt.ylabel("gen")

plt.subplot(1,2,2)
plt.scatter(X[m2, 2], yc[m2, 2], marker=".", alpha=0.2)
plt.plot([-5, 5], [-5, 5], color="black")
plt.xlabel("reco")
plt.ylabel("PF")

In [ ]:
plt.figure(figsize=(8,4))

plt.subplot(1,2,1)
m = y[:, 0] != 0
m2 = yc[:, 0] != 0

plt.scatter(X[m, 3], y[m, 3], marker=".", alpha=0.2)
plt.plot([-4, 4], [-4, 4], color="black")

plt.subplot(1,2,2)
plt.scatter(X[m2, 3], yc[m2, 3], marker=".", alpha=0.2)
plt.plot([-4, 4], [-4, 4], color="black")

In [ ]:
plt.figure(figsize=(8,4))

plt.subplot(1,2,1)
m = y[:, 0] != 0
m2 = yc[:, 0] != 0

plt.scatter(X[m, 4], y[m, 4], marker=".", alpha=0.2)

plt.subplot(1,2,2)
plt.scatter(X[m2, 4], yc[m2, 4], marker=".", alpha=0.2)

In [ ]:
plt.hist(y[m, -1], np.linspace(0,10,11));

In [ ]:
plt.scatter(y[m, 2], y[m, -1], marker=".")

In [ ]:
X[:10]

In [ ]:
np.nonzero(yc[:, 0])

In [ ]:
X[:10, 1]